In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow 

from tensorflow import keras
from keras.layers import Dense

In [ ]:
data=pd.read_csv('../input/eeg-brainwave-dataset-feeling-emotions/emotions.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sum()

In [ ]:
data.shape

In [ ]:
sample=data.loc[0,'fft_0_b':'fft_749_b']
plt.figure(figsize=(16,10))
plt.plot(range(len(sample)), sample)
plt.title("Features fft_0_b through fft_749_b ")
plt.show()

In [ ]:
data['label'].value_counts()

In [ ]:
label_mapping={'NEGATIVE':0,'NEUTRAL':1,'POSITIVE':2}

#preprocessing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def preprocess_inputs(df):
    df=df.copy()
    
    df['label']=df['label'].replace(label_mapping)
    y=df['label'].copy()
    X=df.drop('label',axis=1).copy()
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)
    
    return X_train,X_test,y_train,y_test


In [ ]:
X_train,X_test,y_train,y_test=preprocess_inputs(data)

In [ ]:
X_train.head(3)

Modelling.Use tensorflow for RNN

In [ ]:
inputs=tf.keras.Input(shape=(X_train.shape[1]))
inputs

In [ ]:
inputs=tf.keras.Input(shape=(X_train.shape[1]))

x=tf.keras.layers.Dense(64,activation='relu')(inputs)
x=tf.keras.layers.Dense(64,activation='relu')(x)

outputs=tf.keras.layers.Dense(3,activation='softmax')(x)

model=tf.keras.Model(inputs=inputs,outputs=outputs)

print(model.summary())

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
model_acc=model.evaluate(X_test,y_test,verbose=0)[1]
print("Test Accuracy:{:.3f}".format(model_acc*100))

In [ ]:
y_pred=np.array(list(map(lambda x: np.argmax(x),model.predict(X_test))))
cm=confusion_matrix(y_test,y_pred)
clr=classification_report(y_test,y_pred,target_names=label_mapping.keys())

plt.figure(figsize=(8,8))
sns.heatmap(cm,annot=True,vmin=0,fmt='g',cbar=False,cmap='Blues')
plt.xticks(np.arange(3)+0.5, label_mapping.keys())
plt.yticks(np.arange(3)+0.5, label_mapping.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
print("Classification report\n-------------------------\n",clr)

RNN

In [ ]:
inputs=tf.keras.Input(shape=(X_train.shape[1]))

expand_dims=tf.expand_dims(inputs, axis=2)
gru=tf.keras.layers.GRU(256,return_sequences=True)(expand_dims)
flatten=tf.keras.layers.Flatten()(gru)

outputs=tf.keras.layers.Dense(3,activation='softmax')(flatten)

model=tf.keras.Model(inputs=inputs,outputs=outputs)

print(model.summary())

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
model_acc=model.evaluate(X_test,y_test,verbose=0)[1]
print("Test Accuracy:{:.3f}".format(model_acc*100))

In [ ]:
y_pred=np.array(list(map(lambda x: np.argmax(x),model.predict(X_test))))
cm=confusion_matrix(y_test,y_pred)
clr=classification_report(y_test,y_pred,target_names=label_mapping.keys())

plt.figure(figsize=(8,8))
sns.heatmap(cm,annot=True,vmin=0,fmt='g',cbar=False,cmap='Blues')
plt.xticks(np.arange(3)+0.5, label_mapping.keys())
plt.yticks(np.arange(3)+0.5, label_mapping.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
print("Classification report\n-------------------------\n",clr)